# Tutorial de clasificador lineal perceptrón

## Clasificiación lineal
Una tarea fundamental para el desarrollo de aprendizaje de máquinas es la clasficicación de objetos de acuerdo a sus características. Un ejemplo es la clasificación binaria o lineal; que básicamente consiste en trazar una linea divisoria entre dos conjuntos de datos hasta donde se considera cada elemento en cada grupo más similar con todos los de este grupo que a cualquiera de los elementos del grupo contrario. A veces resulta fácil para un ser humano dividir grupos de cosas intuitivamente, pero para tener resultados precisos y replicables con una máquina es necesario utilizar algorítmos como el perceptron.

El perceptron genera predicciónes a partir de la asignación de pesos a los valores de entrada para calcular un promedio ponderado; si este pasa de cierto límite de acuerdo a una función de activación, entonces se considera que pertenece a uno de los grupos; y, si es menor, al otro grupo.

## Modelo matemático del perceptron
De cada elemento a clasificar, se le asigna, a cada valor $x_{i}$ un peso $\omega_{i}$ de acuerdo a su importancia para la clasificación. Representandolos como vectores $\vec{x}$ y $\vec{\omega}$, y el valor escalar $u$ el límite de activación, se puede describir al perceptron con la ecuación:

$$
f(x) = \begin{cases}
    1, & \text{si } \vec{\omega} \cdot \vec{x} - u > 0 \\
    0, & \text{otro caso}
    \end{cases}
$$


## Bilbioteca scikit-learn

Scikit-learn es una biblioteca para python de código abierto, que añade multiples herramientas de analisis predictivo de datos calificadas como simples, eficientes y accesibles. Utiliza como base las bibliotecas de NumPy, que se utiliza para el procesamiento de números; Scipy, utilizada para el computo científico; y matplotlib, la cual se especializa en la creación de presentaciones gráficas de datos.

Dentro de scikit-learn, encontramos la clase Perceptron en `sklearn.linear_model.Perceptron` la cual se utilizará en este documento para clasificar un dataset.

## Dataset

El dataset a utilizar fué obtenido de un repositorio no oficial que acumula varios dataset de uso común para el entrenamiento y evaluación de sistemas de clasificación donde la fuente especifica no es detallada, pero si menciona 3 diferentes recursos que comforman la colección, y biblioteca de python llamada `common_datasets.

El archivo es presentado con extensión `.dat`siguiendo el formato ARFF de weka. Es un set desbalanceado con 1728 entradas con 6 atribútos que describen las carácterísticas de un automóvil, clasificado según si de considera una _buena compra_ o no. Es esta clasificación lo que nos permite utilizar un perceptron para crear un modelo que nos permita predecir la categoría de un auto en venta.

Los atributos son: costo, mantenimiento, número de puertas, capacidad de personas, tamaño de maletero, y seguridad. Descritos cualitativamente como se verá a continuación.

Para utilizarlo con `sklearn` es necesario convertirlo con `scipy` a un dataframe de `pandas`.

In [1]:
from scipy.io import arff
import pandas as pd

data, meta = arff.loadarff('car-good.dat')
df = pd.DataFrame(data)

# Decode byte strings (ARFF categorical values are loaded as bytes)
df = df.map(lambda x: x.decode('utf-8') if isinstance(x, bytes) else x)

print(df.head(8))

  Buying  Maint Doors Persons Lug_boot Safety     Class
0  vhigh  vhigh     2       2    small    low  negative
1  vhigh  vhigh     2       2    small    med  negative
2  vhigh  vhigh     2       2    small   high  negative
3  vhigh  vhigh     2       2      med    low  negative
4  vhigh  vhigh     2       2      med    med  negative
5  vhigh  vhigh     2       2      med   high  negative
6  vhigh  vhigh     2       2      big    low  negative
7  vhigh  vhigh     2       2      big    med  negative


Entonces, se separan los atributos de las clases

In [2]:
X = df.drop('Class', axis=1)
y = df['Class']

Luego, se tienen que convertir los atributos a valores numéricos para el correcto uso en el perceptrón. Scikit proveé las funciones de `sklearn.preprocessing.OrdinalEncoder` para automatizar el proceso. ChatGPT sugiere el método One-Hot sobre el método ordinal, ya que los atributos clasificados como alto, medio o bajo, so son necesariamente jerárquicos al momento de evaluar si el automóvil es una buena compra en relación a sus atributos. Sin embargo, este separaría cada atributo en varias columnas binarias lo cual hace más difícil interpretar los resultados.

In [3]:
from sklearn.preprocessing import OrdinalEncoder

encoder = OrdinalEncoder()
X_encoded = encoder.fit_transform(X)

## Entrenamiento del perceptrón

Teniendo ya los datos preparados, se procede a separar los datos de entrenamiento y de prueba con `sklearn.model_selection.train_test_split`, para a entrenar el perceptrón.

In [4]:
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.3, random_state=42)

pctron = Perceptron(max_iter=2000, random_state=42)
pctron.fit(X_train, y_train)

print("Pesos aprendidos:", pctron.coef_)
print("Bias aprendido:", pctron.intercept_)

Pesos aprendidos: [[-3. -4. -1.  4.  1.  0.]]
Bias aprendido: [-7.]


## Prueba del perceptrón

In [5]:
def prueba_modelo(perceptron, X):
    return perceptron.predict(X)

predictionsRight = 0
predictionsTotal = 0
for i in range(len(X_encoded)):
    prediccion = prueba_modelo(pctron,[X_encoded[i]])
    print(f"Entrada: {X_encoded[i]}, Salida Esperada: {y[i]}, Salida Predicha: {prediccion}")
    predictionsTotal += 1
    if y[i] in prediccion:
        predictionsRight += 1

Entrada: [3. 3. 0. 0. 2. 1.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 2. 2.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 2. 0.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 1. 1.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 1. 2.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 1. 0.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 0. 1.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 0. 2.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 0. 0. 0.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 1. 2. 1.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 1. 2. 2.], Salida Esperada: negative, Salida Predicha: ['negative']
Entrada: [3. 3. 0. 1. 2. 0.], Salida Espera

## Evaluación del modelo

Gracias a la separación de datos se puede utilizar directamente la función de evaluación de scikit. Pero también se puede lograr manualmente en contra de todas las entradas.

In [6]:
print("Precisión por pruebas: ", pctron.score(X_test, y_test)*100, "%")

print("Precisión del total: ", predictionsRight/predictionsTotal*100, "%")

Precisión por pruebas:  95.37572254335261 %
Precisión del total:  94.96527777777779 %


## Conclusión

Para el conjunto de datos utilizados, el perceptron resultó no ser completamente preciso para todos los atributos. Aún cuando se incrementó el número de iteraciónes, en otras palabras, la solución completa para los datos de entrenamiento se alcanzó dentro de las 1000 iteraciones. Por esto es que la evaluación sobre el total es todavía peor que evaluando solo con los datos de prueba.

## Referencias
GeeksforGeeks. “What Is Perceptron | The Simplest Artificial Neural Network”. 15:38:25+00:00. https://www.geeksforgeeks.org/machine-learning/what-is-perceptron-the-simplest-artificial-neural-network/.

Scikit-Learn. “Perceptron”. Consultado el 7 de noviembre de 2025. https://scikit-learn/stable/modules/generated/sklearn.linear_model.Perceptron.html.

Thapa, Shariph. “Single_Layer_perceptron Datasets”. Kaggle, el 10 de junio de 2024. https://www.kaggle.com/datasets/shariphthapa/single-layer-perceptron-datasets.

Weinberger, Kilian. “Lecture 3: The Perceptron”. 2018. CS4780/CS5780: Machine Learning for Intelligent Systems [FALL 2018]. Consultado el 7 de noviembre de 2025. https://www.cs.cornell.edu/courses/cs4780/2018fa/lectures/lecturenote03.html.
